In [72]:
from dotenv import load_dotenv
import os
import praw
import json

from openai import OpenAI
client = OpenAI()

load_dotenv()

CLIENT_ID_REDDIT = os.getenv("CLIENT_ID_REDDIT")
CLIENT_SECRET_REDDIT = os.getenv("SECRET_KEY_REDDIT")
REDDIT_USERNAME = os.getenv("REDDIT_USERNAME")
REDDIT_PASSWORD = os.getenv("REDDIT_PASSWORD")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

import cohere

co = cohere.ClientV2(api_key=os.getenv("COHERE_API_KEY"))

In [73]:
def crear_reddit():
    reddit = praw.Reddit(
        client_id=CLIENT_ID_REDDIT,
        client_secret=CLIENT_SECRET_REDDIT,
        username=REDDIT_USERNAME,
        password=REDDIT_PASSWORD,
        user_agent="testscript by u/ProudPeace6162"
    )
    return reddit

In [96]:
def generar_instruction_gpt(texto_contextual):
    prompt = (
        f"A partir del siguiente post y comentarios, genera un ejemplo de instruction tuning para un modelo de lenguaje.\n"
        f"Este ejemplo debe estar basado directamente en el contenido del texto entregado y reflejar algún aspecto de la cultura chilena cotidiana, popular o social.\n"
        f"Devuelve directamente un JSON plano, sin usar markdown ni comillas triples.\n"
        f"Debe incluir los campos: 'instruction', 'input' (vacío), y 'output'.\n"
        f"Usa lenguaje claro, directo, sin exagerar.\n\n"
        f"Texto: {texto_contextual}\n"
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un chileno que ayuda a explicar expresiones y situaciones culturales locales sin exagerar."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )

    content = response.choices[0].message.content.strip()

    result = json.loads(content)
    return result

def generar_instruction_cohere_v2(texto_contextual):
    prompt = (
        f"A partir del siguiente post y comentarios, genera un ejemplo de instruction tuning para un modelo de lenguaje.\n"
        f"Este ejemplo debe estar basado directamente en el contenido del texto entregado y reflejar algún aspecto de la cultura chilena cotidiana, popular o social.\n"
        f"Devuelve directamente un JSON plano, sin usar markdown ni comillas triples.\n"
        f"Debe incluir los campos: 'instruction', 'input' (vacío), y 'output'.\n"
        f"Usa lenguaje claro, directo, sin exagerar y no menciones el post.\n\n"
        f"Texto: {texto_contextual}"
    )

    response = co.chat(
        model="command-a-03-2025",
        messages=[
            {"role": "system", "content": "Eres un chileno que explica situaciones culturales locales sin exagerar."},
            {"role": "user", "content": prompt}
        ]
    )

    content = response.message.content[0].text

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print("Error al parsear el JSON:")
        print(content)
        return None

In [99]:
def extraer_contenido_rchile(n):
    reddit = crear_reddit()
    subreddit = reddit.subreddit("chile")
    posts = []
    for post in subreddit.hot(limit=n):
        if post.is_self and post.selftext: # Solo posts de texto
            cuerpo_post = f"Texto: {post.title.strip()} {post.selftext.strip()[:300]}"
            comentarios = []
            post.comments.replace_more(limit=0) # Cargar todos los comentarios
            for comment in post.comments[:5]:
                comentarios.append(comment.body.strip()[:30])
            if comentarios:
                cuerpo_post += "\nComentarios del post:\n" + "\n".join(comentarios)
            posts.append(cuerpo_post)
    return posts

def construir_dataset_instruct(n_ejemplos=10):
    textos = extraer_contenido_rchile(n_ejemplos)
    dataset = []
    for texto in textos:
        resultado = generar_instruction_cohere_v2(texto)
        if resultado['instruction'] and resultado['output']:
            dataset.append(resultado)

    return dataset

In [101]:
dataset = construir_dataset_instruct(n_ejemplos=4)

In [102]:
dataset

[{'instruction': "Explica el uso de la palabra 'talla' en el contexto de la conversación informal en Chile.",
  'input': '',
  'output': "En Chile, la palabra 'talla' se utiliza comúnmente para referirse a una broma, chiste o comentario divertido. En el contexto del comentario, 'las tallas de los simp' se refiere probablemente a las bromas o comentarios humorísticos que hacen las personas consideradas 'simpáticas' o divertidas. Es una forma coloquial de hablar de humor y entretenimiento en conversaciones informales."},
 {'instruction': 'Explica una situación laboral típica en Chile, donde un empleador despide a sus trabajadores pero luego se arrepiente y los reincorpora, generando incertidumbre y posibles conflictos legales.',
  'input': '',
  'output': 'En Chile, es común que las relaciones laborales estén marcadas por la informalidad y la falta de claridad. En este caso, un grupo de trabajadores con contratos indefinidos recibió cartas de despido con un cálculo de indemnización. Sin 